In [187]:
import os
import re
import pandas as pd
from collections import OrderedDict
from datetime import date
from termcolor import colored
import colorama
def AnalyzeVMList(filepath):
    DsList = list()
    try:
        print(colored('Reading file {}'.format(filepath), 'green'))
        with open(filepath) as fp:  
            data=fp.read()
            wid_pattern=re.compile(r'World ID: (\d+)')
            vmname_pattern=re.compile(r'Display Name: (\S+)')
                        
            wid=wid_pattern.findall(data)
            vmname=vmname_pattern.findall(data)
            
            #Translate lists to series
            wid=pd.Series(wid)
            #print(wid)
            vmname=pd.Series(vmname)
            #print(vmname)
            
            WID_VM=pd.DataFrame()
            WID_VM['wid']=wid
            WID_VM['VM']=vmname
            #print(colored('      Dataframe is:', 'green'))
           # print(WID_VM)
            # WID_VM = pd.DataFrame.from_records(se)
            #WID_VM=WID_VM.assign(wid, se.values())
    finally:
        fp.close()
    
    return WID_VM
def DatastoreAnalyse(filepath):
    DS_List=pd.DataFrame()
    print(colored('Reading file {}'.format(filepath), 'green'))
    DS_List = pd.read_fwf(filepath, header=0, usecols=[0, 3], skipinitialspace=True, memory_map=True) #, skiprows=[1])
    
    #print(DS_List)
    return DS_List

def PerfLoad(filepath):
    print(colored('Reading file {}'.format(filepath), 'green'))
    PerfomanceDF=pd.DataFrame()
    FilteredDF=pd.DataFrame()
    NormalizedDF=pd.DataFrame()
    ComposedDF=pd.DataFrame()
    PerfomanceDF=pd.read_csv(filepath, parse_dates=[1], infer_datetime_format=True, quotechar='"', memory_map = True)
    print(colored('Transposing Dataframe', 'green'))
    PerfomanceDF=PerfomanceDF.transpose()
    #Делаем первую строку - заголовком
    PerfomanceDF.columns=PerfomanceDF.iloc[0]
    PerfomanceDF=PerfomanceDF.drop(PerfomanceDF.index[0])
    print(colored('Apply filter to DF', 'green'))
    FilteredDF=PerfomanceDF.filter(like='Disk Per-Device-Per-World', axis=0)
    #Resetting index (Conter names go to data)
    FilteredDF.reset_index(drop=False, inplace=True)
    # Количество колонок len(FilteredDF.columns)
    # Имя колонки print(FilteredDF.columns[1])
    #Возврат пары знчение + время
    print(colored('Stretch Matrix to flat table', 'green'))
    for i in range(len(FilteredDF.columns)-1):
        NormalizedDF=FilteredDF.iloc[:,[0,i+1]]
        NormalizedDF.columns = ['Item', 'Value']
        NormalizedDF['Date']=FilteredDF.columns[i+1]
        ComposedDF=pd.concat([ComposedDF, NormalizedDF], ignore_index=True)

    print(colored('Converting to DateTime', 'green'))
    
    #########################################
    #PostRocessing
    ComposedDF['Date']=pd.to_datetime(ComposedDF['Date'], infer_datetime_format=True)
    

In [234]:
    pd.options.mode.chained_assignment = None
    colorama.init()
    datafolder =  """c:\\Projects\\python\\esxtopanalyzer\\data\\"""
    DatastoreList = datafolder + 'dslist.txt'
    VMList = datafolder + 'vms.txt'
    filepath=datafolder + 'esxc02.csv'
    WID_VM=AnalyzeVMList(VMList)
    DS_List=DatastoreAnalyse(DatastoreList)
    print(colored('Reading file {}'.format(filepath), 'green'))
    PerfomanceDF=pd.DataFrame()
    FilteredDF=pd.DataFrame()
    NormalizedDF=pd.DataFrame()
    ComposedDF=pd.DataFrame()
    PerfomanceDF=pd.read_csv(filepath, parse_dates=[1], infer_datetime_format=True, quotechar='"', memory_map = True)
    print(colored('Transposing Dataframe', 'green'))
    PerfomanceDF=PerfomanceDF.transpose()
    #Делаем первую строку - заголовком
    PerfomanceDF.columns=PerfomanceDF.iloc[0]
    PerfomanceDF=PerfomanceDF.drop(PerfomanceDF.index[0])
    print(colored('Apply filter to DF', 'green'))
    FilteredDF=PerfomanceDF.filter(like='Disk Per-Device-Per-World', axis=0)
    #Resetting index (Conter names go to data)
    FilteredDF.reset_index(drop=False, inplace=True)
    # Количество колонок len(FilteredDF.columns)
    # Имя колонки print(FilteredDF.columns[1])
    #Возврат пары знчение + время
    print(colored('Stretch Matrix to flat table', 'green'))
    for i in range(len(FilteredDF.columns)-1):
        NormalizedDF=FilteredDF.iloc[:,[0,i+1]]
        NormalizedDF.columns = ['Item', 'Value']
        NormalizedDF['Date']=FilteredDF.columns[i+1]
        ComposedDF=pd.concat([ComposedDF, NormalizedDF], ignore_index=True)

    print(colored('Converting to DateTime', 'green'))
    
    #########################################
    #PostRocessing
    ComposedDF['Date']=pd.to_datetime(ComposedDF['Date'], infer_datetime_format=True)
    print(colored('Splitting columns', 'green'))
    ComposedDF['Host']=ComposedDF['Item'].str.split('\\').str.get(2)
    ComposedDF['Parent']=ComposedDF['Item'].str.split('\\').str.get(3)
    ComposedDF['Counter']=ComposedDF['Item'].str.split('\\').str.get(4)
    ComposedDF['DeviceID']=ComposedDF['Parent'].str.split('\(').str.get(1).str.replace('\)','')
    ComposedDF['WorldID']=ComposedDF['DeviceID'].str.split('-').str.get(1)

Reading file c:\Projects\python\esxtopanalyzer\data\vms.txt
Reading file c:\Projects\python\esxtopanalyzer\data\dslist.txt
Reading file c:\Projects\python\esxtopanalyzer\data\esxc02.csv
Transposing Dataframe
Apply filter to DF
Stretch Matrix to flat table
Converting to DateTime
Splitting columns


In [264]:
ComposedDF['DeviceID']=ComposedDF['DeviceID'].str.split('-').str.get(0)

In [265]:
ComposedDF

,Item,Value,Date,Host,Parent,Counter,DeviceID,WorldID,VM
0,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,1,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Device Q Depth,eui.00a0504658335330,68715,NaN
1,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,World Q Depth,eui.00a0504658335330,68715,NaN
2,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Active Commands,eui.00a0504658335330,68715,NaN
3,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Queued Commands,eui.00a0504658335330,68715,NaN
4,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,% Used,eui.00a0504658335330,68715,NaN
5,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Load,eui.00a0504658335330,68715,NaN
6,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Commands/sec,eui.00a0504658335330,68715,NaN
7,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Reads/sec,eui.00a0504658335330,68715,NaN
8,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Writes/sec,eui.00a0504658335330,68715,NaN
9,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,MBytes Read/sec,eui.00a0504658335330,68715,NaN


In [252]:
ComposedDF

,Item,Value,Date,Host,Parent,Counter,DeviceID,WorldID,VM
0,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,1,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Device Q Depth,eui.00a0504658335330-68715,68715,NaN
1,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,World Q Depth,eui.00a0504658335330-68715,68715,NaN
2,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Active Commands,eui.00a0504658335330-68715,68715,NaN
3,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Queued Commands,eui.00a0504658335330-68715,68715,NaN
4,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,% Used,eui.00a0504658335330-68715,68715,NaN
5,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Load,eui.00a0504658335330-68715,68715,NaN
6,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Commands/sec,eui.00a0504658335330-68715,68715,NaN
7,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Reads/sec,eui.00a0504658335330-68715,68715,NaN
8,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Writes/sec,eui.00a0504658335330-68715,68715,NaN
9,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,MBytes Read/sec,eui.00a0504658335330-68715,68715,NaN


In [266]:
ComposedDF.join(DS_List.set_index('Device Name'),on='DeviceID')

,Item,Value,Date,Host,Parent,Counter,DeviceID,WorldID,VM,Volume Name
0,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,1,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Device Q Depth,eui.00a0504658335330,68715,NaN,NaN
1,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,World Q Depth,eui.00a0504658335330,68715,NaN,NaN
2,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Active Commands,eui.00a0504658335330,68715,NaN,NaN
3,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Queued Commands,eui.00a0504658335330,68715,NaN,NaN
4,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,% Used,eui.00a0504658335330,68715,NaN,NaN
5,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Load,eui.00a0504658335330,68715,NaN,NaN
6,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Commands/sec,eui.00a0504658335330,68715,NaN,NaN
7,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Reads/sec,eui.00a0504658335330,68715,NaN,NaN
8,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,Writes/sec,eui.00a0504658335330,68715,NaN,NaN
9,\\mow03-esxc02\Physical Disk Per-Device-Per-Wo...,0,2018-07-03 16:11:37,mow03-esxc02,Physical Disk Per-Device-Per-World(eui.00a0504...,MBytes Read/sec,eui.00a0504658335330,68715,NaN,NaN
